# Prescription Parser using CRF
A Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
('Take', 'Method') = ('2', 'Qty') 

('tablets', 'Form') = ('once', 'Frequency')

('a', 'Period') = ('day', 'PeriodUnit')

('10', 'Duration') = ('days', 'DurationUnit') 

#### Install necesaary library

In [1]:
!pip install sklearn_crfsuite

In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [3]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [4]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [5]:
def tuples_maker(inp, out):
  sample_data = []
  sentences = zip(inp, out)
  sentences = list(sentences)
  for sentence in sentences:
    new_tuples = list(zip(sentence[0], sentence[1]))
    sample_data.append(new_tuples)
  return sample_data

In [6]:
tuples_maker_output = tuples_maker(input_sigs, output_labels)
tuples_maker_output

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [7]:
def triples_maker(whole_data):
  sample_data = []
  for data in whole_data:
    new_tuples = []

    for tuples in data:
      tagged_word = nltk.pos_tag([tuples[0]])
      tagged_word = tagged_word[0][1]
      new_tuple = (tuples[0], tagged_word, tuples[1])
      new_tuples.append(new_tuple)

    sample_data.append(new_tuples)

  return sample_data

In [8]:
sample_data = triples_maker(tuples_maker_output)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'NN', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'NN', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'NN', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)

Using SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag

In [9]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [10]:
def get_features(doc):
  return [token_to_features(doc, i) for i in range (len(doc))]

# A function for generating the list of labels for each document
def get_labels(doc):
  return [label for (token, postag, label) in doc]

X = [get_features (doc) for doc in sample_data]
y = [get_labels (doc) for doc in sample_data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
X_train[0][0]

['bias',
 'word.lower=every',
 'word[-3:]=ery',
 'word[-2:]=ry',
 'word.isupper=False',
 'word.istitle=False',
 'word.isdigit=False',
 'postag=DT',
 'BOS',
 '+1:word.lower=6',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:word.isdigit=True',
 '+1:postag=CD']

In [12]:
y_train[0][0]

'EVERY'

### Training the CRF model with the features extracted using the feature extractor method

In [13]:
import sklearn_crfsuite
from sklearn_crfsuite import CRF, metrics
import pickle

# crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)
crf = CRF(algorithm='lbfgs', max_iterations=50)

# Submit training data to the trainer
crf.fit(X_train, y_train)

# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
filename = 'clf_model.sav'
pickle.dump(crf, open(filename, 'wb'))

In [14]:
labels = crf.classes_
labels

['EVERY',
 'Period',
 'PeriodUnit',
 'Qty',
 'Form',
 'Frequency',
 'Method',
 'FOR',
 'Duration',
 'DurationUnit',
 'TIMES',
 'AND',
 'WITH',
 'FOOD',
 'TO',
 'PeriodMax',
 'BEFORE',
 'WHEN',
 'AFTER',
 'PO',
 'Q4-6H',
 'Q46H',
 'M',
 'THEN',
 'AT',
 'BID',
 'QID',
 'Q6H',
 'DAILY',
 'BY',
 'TID']

### Predicting the test data with the built model

In [15]:
y_pred = crf.predict(X_test)
labels = list(crf.classes_)
f1_score = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels, zero_division=False)
print("F-Score:", f1_score)

F-Score: 0.8660733206187752


### Putting all the prediction logic inside a predict method

In [16]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    print(sig)
    predictions = crf.predict([sig.split(' ')])
    print(predictions)
    return predictions

### Sample predictions

In [17]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
[['FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit']]


In [18]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [19]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [20]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
[['FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit']]


In [21]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
[['FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit']]


In [22]:
predictions = predict("x 30 days")

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [23]:
predictions = predict("x 20 months")

x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [24]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [25]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [26]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form']]


In [27]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
[['FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit']]


In [28]:
predictions = predict("inject 3 units orally")

inject 3 units orally
[['Method', 'Qty', 'Form', 'Frequency']]


In [29]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form']]


In [30]:
predictions = predict("by mouth take three caps")

by mouth take three caps
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form']]


In [31]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
[['FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [32]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
[['FOR', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [33]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [34]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
[['FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit']]


In [35]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [36]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
[['FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [37]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
[['Method', 'Qty', 'Form', 'Frequency']]


In [38]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
[['FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit', 'FOR', 'Duration', 'DurationUnit']]


In [39]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
[['FOR', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [40]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
[['FOR', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [41]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
[['FOR', 'Duration', 'DurationUnit']]


In [42]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form']]


In [43]:
prediction = predict("take two to four tabs")

take two to four tabs
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form']]


In [44]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [45]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [46]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [47]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [48]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [49]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [50]:
prediction = predict("every 60 mins")

every 60 mins
[['FOR', 'Duration', 'DurationUnit']]


In [51]:
prediction = predict("every 10 mins")

every 10 mins
[['FOR', 'Duration', 'DurationUnit']]


In [52]:
prediction = predict("every two to four months")

every two to four months
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form']]


In [53]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [54]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [55]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [56]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [57]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [58]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form']]


In [59]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
[['FOR', 'Duration', 'DurationUnit', 'Qty', 'Form']]


In [60]:
prediction = predict("10 tabs")

10 tabs
[['Qty', 'Form']]
